In [1]:
# https://arxiv.org/abs/cond-mat/0703314
# https://arxiv.org/abs/0808.3779

# https://github.com/scipy/scipy/issues/7151
# https://apps.dtic.mil/sti/pdfs/AD1004183.pdf
# https://www.codeproject.com/Articles/21282/Compute-Permanent-of-a-Matrix-with-Ryser-s-Algorit

# https://rosettacode.org/wiki/Determinant_and_permanent
# https://codegolf.stackexchange.com/questions/97060/calculate-the-permanent-as-quickly-as-possible

# https://stackoverflow.com/questions/38738835/generating-gray-codes
# https://qiita.com/b1ueskydragon/items/75cfee42541ea723080c

# https://qiita.com/phdax/items/3064de264c7933bab2f5
# https://web.archive.org/web/20190108235115/https://www.hackersdelight.org/hdcodetxt/pop.c.txt
# http://graphics.stanford.edu/~seander/bithacks.html#CountBitsSetParallel
# https://stackoverflow.com/questions/9829578/fast-way-of-counting-non-zero-bits-in-positive-integer

# https://stackoverflow.com/questions/22227595/convert-integer-to-binary-array-with-suitable-padding

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import time

In [3]:
# https://github.com/XanaduAI/thewalrus/blob/master/thewalrus/_permanent.py

@jit(nopython=True)
def perm_bbfg(M):  # pragma: no cover
    """
    Returns the permanent of a matrix using the bbfg formula in Gray ordering
    The code is a re-implementation from a Python 2 code found in
    `Permanent code golf
    <https://codegolf.stackexchange.com/questions/97060/calculate-the-permanent-as-quickly-as-possible>`_
    using Numba.
    Args:
        M (array) : a square array.
    Returns:
        float or complex: the permanent of a matrix ``M``
    """

    n = len(M)
    if n == 0:
        return M.dtype.type(1.0)
    row_comb = np.sum(M, 0)
    total = 0
    old_gray = 0
    sign = +1
    binary_power_dict = np.array([2**i for i in range(n)])
    num_loops = 2 ** (n - 1)
    for bin_index in range(1, num_loops + 1):
        reduced = np.prod(row_comb)
        total += sign * reduced
        new_gray = bin_index ^ (bin_index // 2)
        gray_diff = old_gray ^ new_gray
        gray_diff_index = np.searchsorted(binary_power_dict,gray_diff)
        new_vector = M[gray_diff_index]
        direction = 2 * ((old_gray > new_gray) - (old_gray < new_gray))
        for i in range(n):
            row_comb[i] += new_vector[i] * direction
        sign = -sign
        old_gray = new_gray
    return total / num_loops

In [4]:
@jit(nopython=True)
def calc_rsks(L):
    rs = np.arange(1,L+1) ## from 1 to L
    ks = np.linspace(np.pi/(L+1.0),L*np.pi/(L+1.0),L)
    return rs,ks

@jit(nopython=True)
def calc_eks(J,ks):
    return -2.0*J*np.cos(ks)

@jit(nopython=True)
def calc_xkrs(L,ks,rs):
    return np.sqrt(2.0/(L+1.0))*np.sin(np.outer(ks,rs))

@jit(nopython=True)
def calc_ys(t,eks,xkrs):
    xc = (1.0+0.0j)*xkrs # for numba
    expeks = np.diag(np.exp(1j*t*eks))
    return xc @ expeks @ xc

# @jit(nopython=True)
# def calc_fs(ys,system):
#     diag = (1.0+0.0j)*np.diag(system)
#     return ys @ diag @ ys.T.conjugate()

@jit(nopython=True)
def calc_fs_cdw(ys,system,period=1):
    diag = (1.0+0.0j)*np.diag(system)
    ycuts = np.copy(ys[::period,:]) ## avoid NumbaPerformanceWarning to make contiguous arrays by deep copy
#    ycuts = ys[::period,:]
    return ycuts @ diag @ ycuts.T.conjugate()

@jit(nopython=True)
def calc_z(L,system,t,eks,xkrs,period=1):
    ys = calc_ys(t,eks,xkrs)
#    fs = calc_fs(ys,system)
    fs = calc_fs_cdw(ys,system,period=period)
    dlt = np.eye(fs.shape[0],dtype=np.complex128)
    return np.vstack( (np.hstack((fs,dlt-fs)), np.hstack((dlt-fs,fs))) )

@jit(nopython=True)
def calc_z_2(L,system,t,eks,xkrs,period=1):
    ys = calc_ys(t,eks,xkrs)
#    fs = calc_fs(ys,system)
    fs = calc_fs_cdw(ys,system,period=period)
    dlt = np.eye(fs.shape[0],dtype=np.complex128)
    return np.vstack( (np.hstack((fs,dlt-fs)), np.hstack((dlt-fs,fs))) ), fs

@jit(nopython=True)
def get_dat(L,system,J,ts,period=1):
    rs, ks = calc_rsks(L)
    eks = calc_eks(J,ks)
    xkrs = calc_xkrs(L,ks,rs)
    data = np.zeros((len(ts),2),dtype=np.float64)
    for i,t in enumerate(ts):
        matz = calc_z(L,system,t,eks,xkrs,period=period)
        data[i] = np.array([t,-np.log(perm_bbfg(matz).real)])
    return data

In [5]:
## Mott insulator initial
## matrix size: 2L*2L
## len(matz[0,:]) = 2L

period = 1
J = 1.0
#ts = np.linspace(0,20.0,201)
ts = [0.0,10.0,20.0]
#Ls = [2,4,6,8,10,12,14,16,18,20]
Ls = np.arange(2,42,2)
#Ls = [2,4,6]

for i,L in enumerate(Ls):
    rs, ks = calc_rsks(L)
    eks = calc_eks(J,ks)
    xkrs = calc_xkrs(L,ks,rs)
    system = np.array([1.0 if i<L//2 else 0.0 for i in range(L)])
    for j,t in enumerate(ts):
        #matz = calc_z(L,system,t,eks,xkrs,period=period)
        matz, fs = calc_z_2(L,system,t,eks,xkrs,period=period)
        print(L,t,np.linalg.eigvalsh(fs))
        #print("fs",fs)
        #print("fs@fs",fs@fs)
        #print("fs@fs~fs?",np.allclose(fs@fs,fs))

2 0.0 [0. 1.]
2 10.0 [-2.77555756e-17  1.00000000e+00]
2 20.0 [-2.77555756e-17  1.00000000e+00]
4 0.0 [-1.93116402e-17  1.02578367e-16  1.00000000e+00  1.00000000e+00]
4 10.0 [-8.52165425e-17  1.12972118e-16  1.00000000e+00  1.00000000e+00]
4 20.0 [2.62963052e-17 1.40237149e-16 1.00000000e+00 1.00000000e+00]
6 0.0 [-1.70394810e-16  3.16169322e-17  1.94289029e-16  1.00000000e+00
  1.00000000e+00  1.00000000e+00]
6 10.0 [-2.30669576e-16 -1.82256355e-17  7.77316617e-17  1.00000000e+00
  1.00000000e+00  1.00000000e+00]
6 20.0 [-1.38208179e-16 -6.35664779e-17  5.85043600e-17  1.00000000e+00
  1.00000000e+00  1.00000000e+00]
8 0.0 [-3.80621419e-17  3.56090236e-18  4.28496792e-17  2.64895219e-16
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00]
8 10.0 [-1.52210513e-16 -3.99889172e-17  4.75366102e-17  2.43544185e-16
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00]
8 20.0 [-1.42294820e-16 -1.05133894e-16  1.11043411e-16  2.30542121e-16
  1.00000000e+00  1.000000

In [6]:
## Mott insulator initial
## matrix size: 2L*2L
## len(matz[0,:]) = 2L

period = 1
J = 1.0
#ts = np.linspace(0,20.0,201)
ts = [0.0,10.0,20.0]
#Ls = [2,4,6,8,10,12,14,16,18,20]
Ls = np.arange(2,42,2)
#Ls = [2,4,6]

for i,L in enumerate(Ls):
    rs, ks = calc_rsks(L)
    eks = calc_eks(J,ks)
    xkrs = calc_xkrs(L,ks,rs)
    system = np.array([1.0 if i<L//2 else 0.0 for i in range(L)])
    for j,t in enumerate(ts):
        #matz = calc_z(L,system,t,eks,xkrs,period=period)
        matz, fs = calc_z_2(L,system,t,eks,xkrs,period=period)
        #print(L,t,np.linalg.eigvalsh(fs))
        print(L,t)
        #print("fs",fs)
        #print("fs@fs",fs@fs)
        print("fs@fs~fs?",np.allclose(fs@fs,fs))

2 0.0
fs@fs~fs? True
2 10.0
fs@fs~fs? True
2 20.0
fs@fs~fs? True
4 0.0
fs@fs~fs? True
4 10.0
fs@fs~fs? True
4 20.0
fs@fs~fs? True
6 0.0
fs@fs~fs? True
6 10.0
fs@fs~fs? True
6 20.0
fs@fs~fs? True
8 0.0
fs@fs~fs? True
8 10.0
fs@fs~fs? True
8 20.0
fs@fs~fs? True
10 0.0
fs@fs~fs? True
10 10.0
fs@fs~fs? True
10 20.0
fs@fs~fs? True
12 0.0
fs@fs~fs? True
12 10.0
fs@fs~fs? True
12 20.0
fs@fs~fs? True
14 0.0
fs@fs~fs? True
14 10.0
fs@fs~fs? True
14 20.0
fs@fs~fs? True
16 0.0
fs@fs~fs? True
16 10.0
fs@fs~fs? True
16 20.0
fs@fs~fs? True
18 0.0
fs@fs~fs? True
18 10.0
fs@fs~fs? True
18 20.0
fs@fs~fs? True
20 0.0
fs@fs~fs? True
20 10.0
fs@fs~fs? True
20 20.0
fs@fs~fs? True
22 0.0
fs@fs~fs? True
22 10.0
fs@fs~fs? True
22 20.0
fs@fs~fs? True
24 0.0
fs@fs~fs? True
24 10.0
fs@fs~fs? True
24 20.0
fs@fs~fs? True
26 0.0
fs@fs~fs? True
26 10.0
fs@fs~fs? True
26 20.0
fs@fs~fs? True
28 0.0
fs@fs~fs? True
28 10.0
fs@fs~fs? True
28 20.0
fs@fs~fs? True
30 0.0
fs@fs~fs? True
30 10.0
fs@fs~fs? True
30 20.0
fs@fs~f

In [7]:
## Mott insulator initial
## matrix size: 2L*2L
## len(matz[0,:]) = 2L

period = 1
J = 1.0
#ts = np.linspace(0,20.0,201)
ts = [0.0,10.0,20.0]
#Ls = [2,4,6,8,10,12,14,16,18,20]
#Ls = np.arange(2,42,2)
Ls = [2,4,6]

for i,L in enumerate(Ls):
    rs, ks = calc_rsks(L)
    eks = calc_eks(J,ks)
    xkrs = calc_xkrs(L,ks,rs)
    system = np.array([1.0 if i<L//2 else 0.0 for i in range(L)])
    for j,t in enumerate(ts):
        #matz = calc_z(L,system,t,eks,xkrs,period=period)
        matz, fs = calc_z_2(L,system,t,eks,xkrs,period=period)
        #print(L,t,np.linalg.eigvalsh(fs))
        print(L,t)
        print("fs",fs)
        print("fs@fs",fs@fs)
        #print("fs@fs~fs?",np.allclose(fs@fs,fs))

2 0.0
fs [[1.00000000e+00+0.j 1.11022302e-16+0.j]
 [1.11022302e-16+0.j 1.23259516e-32+0.j]]
fs@fs [[1.00000000e+00+0.j 1.11022302e-16+0.j]
 [1.11022302e-16+0.j 1.23259516e-32+0.j]]
2 10.0
fs [[ 7.04041031e-01+0.j         -4.70738517e-17+0.45647263j]
 [-4.70738517e-17-0.45647263j  2.95958969e-01+0.j        ]]
fs@fs [[ 7.04041031e-01+0.j         -4.70738517e-17+0.45647263j]
 [-4.70738517e-17-0.45647263j  2.95958969e-01+0.j        ]]
2 20.0
fs [[ 1.66530969e-01+0.j         -2.98424957e-16+0.37255658j]
 [-2.98424957e-16-0.37255658j  8.33469031e-01+0.j        ]]
fs@fs [[ 1.66530969e-01+0.j         -2.98424957e-16+0.37255658j]
 [-2.98424957e-16-0.37255658j  8.33469031e-01+0.j        ]]
4 0.0
fs [[ 1.00000000e+00+0.j -5.55111512e-17+0.j  2.77555756e-17+0.j
   8.32667268e-17+0.j]
 [-5.55111512e-17+0.j  1.00000000e+00+0.j -7.70371978e-34+0.j
  -1.11022302e-16+0.j]
 [ 2.77555756e-17+0.j -7.70371978e-34+0.j  7.70371978e-34+0.j
   2.31111593e-33+0.j]
 [ 8.32667268e-17+0.j -1.11022302e-16+0.j  2.31

In [8]:
## Mott insulator initial
## matrix size: 2L*2L
## len(matz[0,:]) = 2L

period = 1
J = 1.0
#ts = np.linspace(0,20.0,201)
ts = [0.0,10.0,20.0]
#Ls = [2,4,6,8,10,12,14,16,18,20]
Ls = np.arange(2,42,2)
#Ls = [2,4,6]

for i,L in enumerate(Ls):
    rs, ks = calc_rsks(L)
    eks = calc_eks(J,ks)
    xkrs = calc_xkrs(L,ks,rs)
    system = np.array([1.0 if i<L//2 else 0.0 for i in range(L)])
    for j,t in enumerate(ts):
        #matz = calc_z(L,system,t,eks,xkrs,period=period)
        matz, fs = calc_z_2(L,system,t,eks,xkrs,period=period)
        #print(L,t,np.linalg.eigvalsh(fs))
        print(L,t)
        print("fs~fs.conjugat().T?",np.allclose(fs,fs.conjugate().T))

2 0.0
fs~fs.conjugat().T? True
2 10.0
fs~fs.conjugat().T? True
2 20.0
fs~fs.conjugat().T? True
4 0.0
fs~fs.conjugat().T? True
4 10.0
fs~fs.conjugat().T? True
4 20.0
fs~fs.conjugat().T? True
6 0.0
fs~fs.conjugat().T? True
6 10.0
fs~fs.conjugat().T? True
6 20.0
fs~fs.conjugat().T? True
8 0.0
fs~fs.conjugat().T? True
8 10.0
fs~fs.conjugat().T? True
8 20.0
fs~fs.conjugat().T? True
10 0.0
fs~fs.conjugat().T? True
10 10.0
fs~fs.conjugat().T? True
10 20.0
fs~fs.conjugat().T? True
12 0.0
fs~fs.conjugat().T? True
12 10.0
fs~fs.conjugat().T? True
12 20.0
fs~fs.conjugat().T? True
14 0.0
fs~fs.conjugat().T? True
14 10.0
fs~fs.conjugat().T? True
14 20.0
fs~fs.conjugat().T? True
16 0.0
fs~fs.conjugat().T? True
16 10.0
fs~fs.conjugat().T? True
16 20.0
fs~fs.conjugat().T? True
18 0.0
fs~fs.conjugat().T? True
18 10.0
fs~fs.conjugat().T? True
18 20.0
fs~fs.conjugat().T? True
20 0.0
fs~fs.conjugat().T? True
20 10.0
fs~fs.conjugat().T? True
20 20.0
fs~fs.conjugat().T? True
22 0.0
fs~fs.conjugat().T? True


In [9]:
## Mott insulator initial
## matrix size: 2L*2L
## len(matz[0,:]) = 2L

period = 1
J = 1.0
#ts = np.linspace(0,20.0,201)
ts = [0.0,10.0,20.0]
#Ls = [2,4,6,8,10,12,14,16,18,20]
Ls = np.arange(2,42,2)
#Ls = [2,4,6]

for i,L in enumerate(Ls):
    rs, ks = calc_rsks(L)
    eks = calc_eks(J,ks)
    xkrs = calc_xkrs(L,ks,rs)
    system = np.array([1.0 if i<L//2 else 0.0 for i in range(L)])
    for j,t in enumerate(ts):
        #matz = calc_z(L,system,t,eks,xkrs,period=period)
        matz, fs = calc_z_2(L,system,t,eks,xkrs,period=period)
        #print(L,t,np.linalg.eigvalsh(fs))
        print(L,t)
        print("fs@fs.conjugat().T~I?",np.allclose(fs@fs.conjugate().T,np.eye(fs.shape[0])))

2 0.0
fs@fs.conjugat().T~I? False
2 10.0
fs@fs.conjugat().T~I? False
2 20.0
fs@fs.conjugat().T~I? False
4 0.0
fs@fs.conjugat().T~I? False
4 10.0
fs@fs.conjugat().T~I? False
4 20.0
fs@fs.conjugat().T~I? False
6 0.0
fs@fs.conjugat().T~I? False
6 10.0
fs@fs.conjugat().T~I? False
6 20.0
fs@fs.conjugat().T~I? False
8 0.0
fs@fs.conjugat().T~I? False
8 10.0
fs@fs.conjugat().T~I? False
8 20.0
fs@fs.conjugat().T~I? False
10 0.0
fs@fs.conjugat().T~I? False
10 10.0
fs@fs.conjugat().T~I? False
10 20.0
fs@fs.conjugat().T~I? False
12 0.0
fs@fs.conjugat().T~I? False
12 10.0
fs@fs.conjugat().T~I? False
12 20.0
fs@fs.conjugat().T~I? False
14 0.0
fs@fs.conjugat().T~I? False
14 10.0
fs@fs.conjugat().T~I? False
14 20.0
fs@fs.conjugat().T~I? False
16 0.0
fs@fs.conjugat().T~I? False
16 10.0
fs@fs.conjugat().T~I? False
16 20.0
fs@fs.conjugat().T~I? False
18 0.0
fs@fs.conjugat().T~I? False
18 10.0
fs@fs.conjugat().T~I? False
18 20.0
fs@fs.conjugat().T~I? False
20 0.0
fs@fs.conjugat().T~I? False
20 10.0
fs@fs.

In [10]:
## Mott insulator initial
## matrix size: 2L*2L
## len(matz[0,:]) = 2L

period = 1
J = 1.0
#ts = np.linspace(0,20.0,201)
ts = [0.0,10.0,20.0]
#Ls = [2,4,6,8,10,12,14,16,18,20]
Ls = np.arange(2,42,2)
#Ls = [2,4,6]

for i,L in enumerate(Ls):
    rs, ks = calc_rsks(L)
    eks = calc_eks(J,ks)
    xkrs = calc_xkrs(L,ks,rs)
    system = np.array([1.0 if i<L//2 else 0.0 for i in range(L)])
    for j,t in enumerate(ts):
        #matz = calc_z(L,system,t,eks,xkrs,period=period)
        matz, fs = calc_z_2(L,system,t,eks,xkrs,period=period)
        #print(L,t,np.linalg.eigvalsh(fs))
        print(L,t)
        print("matz@matz.conjugat().T~I?",np.allclose(matz@matz.conjugate().T,np.eye(matz.shape[0])))

2 0.0
matz@matz.conjugat().T~I? True
2 10.0
matz@matz.conjugat().T~I? True
2 20.0
matz@matz.conjugat().T~I? True
4 0.0
matz@matz.conjugat().T~I? True
4 10.0
matz@matz.conjugat().T~I? True
4 20.0
matz@matz.conjugat().T~I? True
6 0.0
matz@matz.conjugat().T~I? True
6 10.0
matz@matz.conjugat().T~I? True
6 20.0
matz@matz.conjugat().T~I? True
8 0.0
matz@matz.conjugat().T~I? True
8 10.0
matz@matz.conjugat().T~I? True
8 20.0
matz@matz.conjugat().T~I? True
10 0.0
matz@matz.conjugat().T~I? True
10 10.0
matz@matz.conjugat().T~I? True
10 20.0
matz@matz.conjugat().T~I? True
12 0.0
matz@matz.conjugat().T~I? True
12 10.0
matz@matz.conjugat().T~I? True
12 20.0
matz@matz.conjugat().T~I? True
14 0.0
matz@matz.conjugat().T~I? True
14 10.0
matz@matz.conjugat().T~I? True
14 20.0
matz@matz.conjugat().T~I? True
16 0.0
matz@matz.conjugat().T~I? True
16 10.0
matz@matz.conjugat().T~I? True
16 20.0
matz@matz.conjugat().T~I? True
18 0.0
matz@matz.conjugat().T~I? True
18 10.0
matz@matz.conjugat().T~I? True
18 20.0

In [11]:
## CDW 010101... initial
## matrix size: L*L
## len(matz[0,:]) = L

period = 2
J = 1.0
#ts = np.linspace(0,20.0,201)
ts = [0.0,10.0,20.0]
#Ls = [2,4,6,8,10,12,14,16,18,20]
#Ls = np.arange(2,42,2)
Ls = [2,12,22]

for i,L in enumerate(Ls):
    rs, ks = calc_rsks(L)
    eks = calc_eks(J,ks)
    xkrs = calc_xkrs(L,ks,rs)
    system = np.array([1.0 if i<L//2 else 0.0 for i in range(L)])
    for j,t in enumerate(ts):
        #matz = calc_z(L,system,t,eks,xkrs,period=period)
        matz, fs = calc_z_2(L,system,t,eks,xkrs,period=period)
        print(L,t,np.linalg.eigvalsh(fs))
        #print("fs",fs)
        #print("fs@fs",fs@fs)
        print("fs@fs~fs?",np.allclose(fs@fs,fs))

2 0.0 [1.]
fs@fs~fs? True
2 10.0 [0.70404103]
fs@fs~fs? False
2 20.0 [0.16653097]
fs@fs~fs? False
12 0.0 [-1.38777878e-16 -8.60953881e-18  8.60953881e-18  1.00000000e+00
  1.00000000e+00  1.00000000e+00]
fs@fs~fs? True
12 10.0 [0.00797866 0.38325206 0.49780131 0.53165344 0.62467276 0.92583576]
fs@fs~fs? False
12 20.0 [0.00633621 0.03672873 0.20314523 0.81894561 0.95529061 0.99315104]
fs@fs~fs? False
22 0.0 [-2.79577177e-16 -5.44108109e-17  1.97472826e-17  5.69830586e-17
  1.51218516e-16  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00  1.00000000e+00  1.00000000e+00]
fs@fs~fs? True
22 10.0 [1.20497117e-09 5.53610563e-03 3.19274823e-02 4.40428943e-01
 4.58947393e-01 5.00132608e-01 5.01855213e-01 5.67267065e-01
 8.20184181e-01 9.56587289e-01 9.99990958e-01]
fs@fs~fs? False
22 20.0 [0.05235704 0.06508021 0.27768484 0.30393295 0.47767393 0.50712838
 0.51570737 0.69099191 0.87280786 0.97055713 0.99672677]
fs@fs~fs? False


In [12]:
## CDW 001001001... initial
## matrix size: 2L/3*2L/3
## len(matz[0,:]) = 2L/3

period = 3
J = 1.0
#ts = np.linspace(0,20.0,201)
ts = [0.0,10.0,20.0]
#Ls = np.arange(3,63,3)
Ls = [3,12,21]

for i,L in enumerate(Ls):
    rs, ks = calc_rsks(L)
    eks = calc_eks(J,ks)
    xkrs = calc_xkrs(L,ks,rs)
    system = np.array([1.0 if i<L//2 else 0.0 for i in range(L)])
    for j,t in enumerate(ts):
        #matz = calc_z(L,system,t,eks,xkrs,period=period)
        matz, fs = calc_z_2(L,system,t,eks,xkrs,period=period)
        print(L,t,np.linalg.eigvalsh(fs))
        #print("fs",fs)
        #print("fs@fs",fs@fs)
        print("fs@fs~fs?",np.allclose(fs@fs,fs))

3 0.0 [1.]
fs@fs~fs? True
3 10.0 [0.24752184]
fs@fs~fs? False
3 20.0 [6.09477761e-10]
fs@fs~fs? True
12 0.0 [-1.13940062e-16 -2.76906231e-17  1.00000000e+00  1.00000000e+00]
fs@fs~fs? True
12 10.0 [0.02635185 0.28081381 0.53704357 0.9195228 ]
fs@fs~fs? False
12 20.0 [0.07350807 0.26503846 0.52445784 0.69182087]
fs@fs~fs? False
21 0.0 [5.93863394e-17 8.92113373e-17 2.78454107e-16 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00]
fs@fs~fs? True
21 10.0 [0.02282851 0.09206439 0.35886378 0.37639023 0.58113069 0.62999568
 0.91375788]
fs@fs~fs? False
21 20.0 [0.24642386 0.38513295 0.4013883  0.57953365 0.68051371 0.7552768
 0.7769729 ]
fs@fs~fs? False
